<a href="https://colab.research.google.com/github/santiago2588/distillation_column_training/blob/main/Soluciones_colab/07_xgboost_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data wrangling
import pandas as pd
import numpy as np

In [2]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm

In [3]:
# Models
from xgboost import XGBRegressor

In [4]:
# Models Pipelines
from sklearn.pipeline import Pipeline

In [5]:
# Install the optuna library
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 13.7 MB/s eta 0:00:00


In [6]:
#Hyperparameter optimization
import optuna

In [7]:
# Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [8]:
# Save model
import joblib

In [9]:
!wget https://raw.githubusercontent.com/santiago2588/distillation_column_training/main/data/transformed_data.csv -O transformed_data.csv

--2025-07-07 14:05:31--  https://raw.githubusercontent.com/santiago2588/distillation_column_training/main/data/transformed_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12344 (12K) [text/plain]
Saving to: ‘transformed_data.csv’

transformed_data.cs 100%[===================>]  12.05K  --.-KB/s    in 0.001s  

2025-07-07 14:05:31 (19.4 MB/s) - ‘transformed_data.csv’ saved [12344/12344]



In [10]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("transformed_data.csv")

# Display the first few rows to confirm it loaded correctly
df.head()

,PressureC1_diff,FlowC1,Temp1,Yield
0,0.0000,432.0636,139.9857,69.400623
1,-9.9628,487.4029,131.0470,66.532666
2,-0.0695,437.3516,118.2666,71.102193
3,0.2257,481.8314,118.1769,69.793481
4,-0.1678,412.6471,120.7891,71.489516


In [11]:
# Define features (X) and target variable (y)
X = df.drop('Yield', axis=1)  # Features (all columns except 'Job Offer')
y = df['Yield']  # Target variable

In [12]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (202, 3)
Testing data shape: (51, 3)


In [13]:
def get_metrics(y_true, y_pred):

    dict_metrics = {
        'MSE': mean_squared_error(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2 Score': r2_score(y_true, y_pred),

    }
    return dict_metrics

In [14]:
# Create model with XGBRegressor
pip_model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)

# Fit pipeline
pip_model_xgb.fit(X_train, y_train)

# Generate predictions
y_pred = pip_model_xgb.predict(X_test)

# Evaluate metrics
metrics = get_metrics(y_test, y_pred)

# View results
print(metrics)

{'MSE': 0.5043979371243295, 'MAE': 0.5494068029021147, 'R2 Score': 0.7527133663152991}


In [15]:
#Hyperparameter optimization with Optuna

# Define the objective function
def objective(trial):
    # Define the hyperparameter search space
    param = {
        "verbosity": 0,
        "objective": "reg:squarederror",  # For regression tasks
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "eta": trial.suggest_float("eta", 1e-3, 0.3, log=True),
        "gamma": trial.suggest_float("gamma", 1e-3, 10.0, log=True),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    }

    # Create and train model
    model = XGBRegressor(**param)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate error (you can change this to other metrics)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Create and run study
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

# Add this to suppress Optuna logging
optuna.logging.set_verbosity(optuna.logging.WARNING)

# When optimizing, disable progress bar if desired
study.optimize(objective, n_trials=50, show_progress_bar=False)

# Train final model with best parameters
best_params = study.best_trial.params
print("Best hyperparameters:", study.best_params)

[I 2025-07-07 14:05:57,364] A new study created in memory with name: no-name-0fa48268-22a8-4fdb-8bb8-f4d10e44fee1


Best hyperparameters: {'booster': 'dart', 'lambda': 0.005069962404388357, 'alpha': 0.003962303952263623, 'max_depth': 7, 'eta': 0.060919828834511566, 'gamma': 0.16162397858876598, 'grow_policy': 'lossguide', 'subsample': 0.5550613125781216, 'colsample_bytree': 0.7133456145286302, 'min_child_weight': 3}


In [16]:
# Train the final model
final_model = XGBRegressor(**best_params)
final_model.fit(X_train, y_train)

# Log metrics
y_pred = final_model.predict(X_test)
metrics = get_metrics(y_test, y_pred)

# View results
print("Metrics on test set:", metrics)

# You can optionally save the model using joblib or pickle if needed without MLflow
# joblib.dump(final_model, 'final_xgboost_model.joblib')

Metrics on test set: {'MSE': 0.48899161633602556, 'MAE': 0.5209982432042745, 'R2 Score': 0.7602664844484275}


In [17]:
# Visualize the optimization history
optuna.visualization.plot_optimization_history(study).show()

In [18]:
# Visualize parameter importance
optuna.visualization.plot_param_importances(study).show()